# ECP 2019 vs 2023 — Notebook con _dropdowns_ internos de año  
Cada gráfica incluye un control desplegable dentro del propio objeto Plotly para alternar entre **2019** y **2023** sin necesidad de ejecutar celdas adicionales.

> **Requisitos**  
> * `pandas`, `plotly`  
> * Archivos `ecp_2019.parquet` y `ecp_2023.parquet` en la carpeta `data`.


In [20]:
import os, pandas as pd, plotly.graph_objects as go, plotly.express as px

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
df2019 = pd.read_parquet(os.path.join(DATA_DIR, 'ecp_2019.parquet'))
df2023 = pd.read_parquet(os.path.join(DATA_DIR, 'ecp_2023.parquet'))

print(f'Filas 2019: {len(df2019):,} | Filas 2023: {len(df2023):,}')


Filas 2019: 43,156 | Filas 2023: 46,392


In [21]:
def porcentaje_ponderado(df, var, valor_si=1):
    if var not in df.columns: return 0.0
    yes = df.loc[df[var]==valor_si,'WEIGHT'].sum()
    tot = df['WEIGHT'].sum()
    return 100*yes/tot if tot else 0

def perc_df_cat(df, var, mapping):
    w = df.groupby(var)['WEIGHT'].sum()
    p = (w/w.sum()*100).reset_index().rename(columns={'WEIGHT':'pct'})
    p[var] = p[var].map(mapping)
    return p

def dropdown_bar(df19, df23, xcol, ycol, title, y_range=None):
    fig = go.Figure()
    fig.add_bar(x=df19[xcol], y=df19[ycol], text=[f'{v:.1f}%' for v in df19[ycol]],
                textposition='auto', marker_line=dict(color='black', width=1),
                name='2019', visible=True)
    fig.add_bar(x=df23[xcol], y=df23[ycol], text=[f'{v:.1f}%' for v in df23[ycol]],
                textposition='auto', marker_line=dict(color='black', width=1),
                name='2023', visible=False)

    # Dropdown
    fig.update_layout(
        updatemenus=[dict(
            buttons=[
                dict(label='2019', method='update',
                     args=[{'visible':[True,False]},
                           {'title':f'{title} — 2019'}]),
                dict(label='2023', method='update',
                     args=[{'visible':[False,True]},
                           {'title':f'{title} — 2023'}]),
            ],
            direction='down', x=0.0, y=1.15
        )],
        title=f'{title} — 2019',
        template='plotly_white',
        yaxis_title='% ponderado'
    )
    if y_range:
        fig.update_yaxes(range=y_range)
    return fig


In [22]:
global_participacion = {1:'Sí votó',2:'No votó',99:'NS/NR'}

group_to_codes = {
    'Animadversión política':['P5336S6','P5336S7','P5336S8','P5336S10','P5336S19','P5336S11'],
    'Dificultad logística':['P5336S14','P5336S15','P5336S17','P5336S13'],
    'Menor/Cédula':['P5336S1','P5336S2'],
    'Otra':['P5336S12']
}

to_vote = {'P5337S1':'Apoyo','P5337S2':'Programa','P5337S3':'Ideología',
           'P5337S4':'Responsabilidad','P5337S5':'Influencia'}

to_difficulties = {'P5338S1':'Mesa lejana','P5338S2':'Colas','P5338S3':'Cédula',
                   'P5338S4':'Horario','P5338S5':'Barreras'}

to_transparency = {'P5339S1':'Municipal','P5339S2':'Departamental','P5339S3':'Nacional'}

labels_id = {1:'Sí',2:'No',99:'NS/NR'}


## 1 · Participación en presidenciales (P6933)

In [23]:
df19 = perc_df_cat(df2019, 'P6933', global_participacion)
df23 = perc_df_cat(df2023, 'P6933', global_participacion)
fig = dropdown_bar(df19, df23, 'P6933', 'pct', 'Participación electoral')
fig.show()


## 2 · Razones agrupadas para **no** votar

In [24]:
def grouped_reason_df(df, group_map):
    tot = df['WEIGHT'].sum()
    rows=[]
    for cat, codes in group_map.items():
        w = sum(df.loc[df[c]==1,'WEIGHT'].sum() for c in codes if c in df.columns)
        rows.append({'cat':cat,'pct':100*w/tot if tot else 0})
    return pd.DataFrame(rows)

df19 = grouped_reason_df(df2019, group_to_codes)
df23 = grouped_reason_df(df2023, group_to_codes)
fig = dropdown_bar(df19, df23, 'cat', 'pct', 'Razones no voto (agrupadas)', y_range=[0,40])
fig.show()


## 3 · Razones para votar

In [25]:
def simple_reason_df(df, mapping):
    return pd.DataFrame({'razón':list(mapping.values()),
                         'pct':[porcentaje_ponderado(df,v) for v in mapping]})

df19 = simple_reason_df(df2019, to_vote)
df23 = simple_reason_df(df2023, to_vote)
fig = dropdown_bar(df19, df23, 'razón', 'pct', 'Razones para votar', y_range=[0,40])
fig.show()


## 4 · Dificultades al votar

In [26]:
df19 = simple_reason_df(df2019, to_difficulties)
df23 = simple_reason_df(df2023, to_difficulties)
fig = dropdown_bar(df19, df23, 'razón', 'pct', 'Dificultades al votar', y_range=[0,40])
fig.show()


## 5 · Transparencia del conteo

In [27]:
df19 = simple_reason_df(df2019, to_transparency)
df23 = simple_reason_df(df2023, to_transparency)
fig = dropdown_bar(df19, df23, 'razón', 'pct', 'Transparencia conteo', y_range=[0,100])
fig.show()


## 6 · Identificación partidista

In [28]:
df19 = perc_df_cat(df2019, 'P5323', labels_id)
df23 = perc_df_cat(df2023, 'P5323', labels_id)
fig = dropdown_bar(df19, df23, 'P5323', 'pct', 'Identificación partidista')
fig.show()


## 7 · Grupos ideológicos

In [32]:
bins=[0,3.5,6.5,10]; cats=['Izquierda','Centro','Derecha']
def ideology_df(df):
    grp = pd.cut(df['P5328'], bins=bins, labels=cats)
    pct = (df.assign(g=grp).groupby('g')['WEIGHT'].sum()/df['WEIGHT'].sum()*100).reset_index()
    pct.columns=['grupo','pct']; return pct

df19 = ideology_df(df2019); df23 = ideology_df(df2023)
fig = dropdown_bar(df19, df23, 'grupo', 'pct', 'Grupos ideológicos', y_range=[0,100])
fig.show()


/tmp/ipykernel_79388/2363978614.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_79388/2363978614.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## 8 · Top 15 variables con más datos faltantes

In [30]:
def miss_df(df):
    return df.isna().mean().mul(100).sort_values(ascending=False).head(15).reset_index().rename(
        columns={'index':'var',0:'pct'}
    )

df19 = miss_df(df2019)
df23 = miss_df(df2023)
fig = dropdown_bar(df19, df23, 'var', 'pct', 'Top 15 faltantes')
fig.update_layout(xaxis_tickangle=-45)
fig.show()
